In [6]:
import requests
import mysql.connector
import time
import discord_notifier


def insert_match_details(match_details):
    try:
        cnx = mysql.connector.connect(user='lol_user', password='your_password',
                                      host='35.203.46.85', database='lol_retention')
        cursor = cnx.cursor()

        add_match = """
        INSERT INTO match_details (match_id, game_creation, game_duration, game_mode, game_type, game_version, 
                                   map_id, participant_id, summoner_id, puuid, champion_id, spell1_id, spell2_id, 
                                   item0, item1, item2, item3, item4, item5, item6, kills, deaths, assists, 
                                   total_damage_dealt, total_damage_taken, gold_earned, vision_score, 
                                   total_minions_killed, team_position, win)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
                %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
            game_creation = VALUES(game_creation),
            game_duration = VALUES(game_duration),
            total_damage_dealt = VALUES(total_damage_dealt),
            total_damage_taken = VALUES(total_damage_taken),
            gold_earned = VALUES(gold_earned),
            vision_score = VALUES(vision_score)
        """
        
        cursor.executemany(add_match, match_details)
        cnx.commit()
        cursor.close()
        cnx.close()
        print(f"Inserted {len(match_details)} match details.")
        
    except mysql.connector.Error as err:
        print("Error:", err)

def fetch_match_details(match_id):
    
    api_key = "RGAPI-a6eedfcb-600e-4f76-b541-0a25e232aa36"
    url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching match details for {match_id}: {response.status_code}")
        return None

def pull_and_insert_match_details():
    # Fetch match_ids from your match_ids table
    try:
        cnx = mysql.connector.connect(user='lol_user', password='your_password',
                                      host='35.203.46.85', database='lol_retention')
        cursor = cnx.cursor()

        # Fetch match_ids that are not in match_details table
        query = """
        SELECT m.match_id FROM match_ids m
        LEFT JOIN match_details md ON m.match_id = md.match_id
        WHERE md.match_id IS NULL
        """
        cursor.execute(query)
        match_ids = [row[0] for row in cursor.fetchall()]
        cursor.close()
        cnx.close()

    except mysql.connector.Error as err:
        print("Error:", err)
        return

    # Process in batches
    batch_size = 100
    val=1
    for i in range(0, len(match_ids), batch_size):
        print(f"times called the batch is: {val}")
        match_details_batch = []
        batch = match_ids[i:i + batch_size]
        
        for match_id in batch:
            time.sleep(1.3)
            print(f"fetching values for match: {match_id}")
            match_data = fetch_match_details(match_id)
            if match_data:
                # Extract relevant data for each participant and append to match_details_batch
                for participant in match_data['info']['participants']:
                    match_details_batch.append((
                        match_id,
                        match_data['info']['gameCreation'],
                        match_data['info']['gameDuration'],
                        match_data['info']['gameMode'],
                        match_data['info']['gameType'],
                        match_data['info']['gameVersion'],
                        match_data['info']['mapId'],
                        participant.get('participantId'),
                        participant.get('summonerId'),
                        participant.get('puuid'),
                        participant.get('championId'),
                        participant.get('spell1Id'),
                        participant.get('spell2Id'),
                        participant.get('item0'),
                        participant.get('item1'),
                        participant.get('item2'),
                        participant.get('item3'),
                        participant.get('item4'),
                        participant.get('item5'),
                        participant.get('item6'),
                        participant.get('kills'),
                        participant.get('deaths'),
                        participant.get('assists'),
                        participant.get('totalDamageDealt'),
                        participant.get('totalDamageTaken'),
                        participant.get('goldEarned'),
                        participant.get('visionScore'),
                        participant.get('totalMinionsKilled'),
                        participant.get('teamPosition', 'UNKNOWN'),
                        participant.get('win')
                    ))

        # Insert batch into DB
        print(f"inserting values in db for bach: {val}")
        insert_match_details(match_details_batch)
        val += 1
        
        
if __name__ == "__main__":
    try: 
        pull_and_insert_match_details()
    except Exception as e:
        print(f"An error occurred: {e}")
    finally: 
        discord_notifier.send_discord_message()
    


times called the batch is: 1
fetching values for match: NA1_5001154509
fetching values for match: NA1_5002701527
fetching values for match: NA1_5004189564
fetching values for match: NA1_5006197541
fetching values for match: NA1_5006908577
fetching values for match: NA1_5007670578
fetching values for match: NA1_5007839714
fetching values for match: NA1_5008108805
fetching values for match: NA1_5008976178
fetching values for match: NA1_5009277639
fetching values for match: NA1_5009576905
fetching values for match: NA1_5011295988
fetching values for match: NA1_5012403989
fetching values for match: NA1_5013173459
fetching values for match: NA1_5013193104
fetching values for match: NA1_5013808124
fetching values for match: NA1_5013837898
fetching values for match: NA1_5013981507
fetching values for match: NA1_5014873477
fetching values for match: NA1_5015490514
fetching values for match: NA1_5015533941
fetching values for match: NA1_5015564680
fetching values for match: NA1_5016072767
fetch